# Basic Setting

In [0]:
from google.colab import files  #google colab上專用
import random
import numpy as np
import math                     #跑自然數字的函示exp()
import csv
import time
import heapq                    #來找一堆最大、小值的index
np.set_printoptions(suppress=True)

upload = files.upload()         #開啟上傳介面並載入(google colab上專用)
ExRate = np.loadtxt(open("SimExRatesFall19.csv", "rb"), delimiter=",", skiprows=1)  #將模擬匯率的CSV轉成matrix[1000*2]
putDM = np.array([[0.66,0.085855],[0.65,0.032191],[0.64,0.020795],
                  [0.63,0.017001],[0.62,0.013711],[0.61,0.010851],
                  [0.60,0.008388],[0.59,0.006291],[0.55,0.001401]])

putBP = np.array([[1.3,0.137213],[1.25,0.082645],[1.2,0.045060],
                  [1.15,0.028348],[1.1,0.016146],[1.05,0.007860],
                  [1.00,0.003277],[0.95,0.001134],[0.90,0.000245]])

n = 18                          #題目的要求，所以此數字不可以調。(numbers of decision variable)

Saving SimExRatesFall19.csv to SimExRatesFall19.csv


# Function A
### Random generate a solution with a set of decision variable and adjusted variable

In [0]:
def generate_a_solution():
  decision_var = []
  adjust_var = []
  for a in range(2):                        #有兩組決策變數，所以loop兩次
    temp_sum = 0
    temp_var = []
    for b in range(9):                      #並且一組為9個變數
      x = random.uniform(0,1)               #使用均勻分配
      temp_var.append(x)                    #將隨機生成的數丟到暫存list裡
      temp_sum = temp_sum + x               #紀錄9個變數的總和，以便於標準化
    for k in temp_var :
      decision_var.append(k/temp_sum)       #將標準化完的變數丟到真正的決策變數list裡
      adjust_var.append(first_adjust_var)   #將相對應數量的調整變數也丟到調整變數list裡
  
  output_var = decision_var+adjust_var      #合併決策變數list、調整變數list
  return(output_var)


# Function B
### Apply recombination and mutation to the parents to produce offspring

In [0]:
def mutation_step1(x,y):
  a = [x,y]
  b = random.sample(a,1)       #b為一個元素的一維陣列
  return b[0]            

In [0]:
def mutation_step2(x,y):
  a = (x+y)/2
  return a

In [0]:
def mutation_step3(x,k):                                                 #k為當輪交配之常態分佈產生的固定值
  a = np.random.standard_normal(1)                                       #a為一個元素的一維陣列
  b = math.exp(mutate_constant*k + mutate_constant*a[0])                 #a[0]這樣表示才會是一個數字
  c = x*b
  return c

In [0]:
def mutation_step4(array_decision,array_adjust):
  temp_sum = 0
  temp_var = []
  output_var = []
  for a in range(9):                                                                 #總共有9個決策變數，所以做9次
    decision_var = array_decision[a] + np.random.normal(0,array_adjust[a],1)[0]    
    while decision_var < 0 :                                                         ###(這個方法可換)當算出來的決策變數(尚未標準化前)小於零，則重新算一次，因為投資比例不能是負的
      decision_var = array_decision[a] + np.random.normal(0,array_adjust[a],1)[0]
    temp_var.append(decision_var)                           
    temp_sum = temp_sum + decision_var
  for b in temp_var :
    output_var.append(b/temp_sum)                                                    #將所有決策變數標準化
  return output_var

In [0]:
def mutation(matrix) : 
  output_var = []
  for i in range(kid_size):                                                     #需要幾個子代就做幾次
    temp_kid_decision_dm =[]
    temp_kid_decision_bp =[]
    kid_adjust_dm = []
    kid_adjust_bp = []
    indexList = range(pop_size)                                                 #隨機抽出兩個當父母-1
    randomIndex = random.sample(indexList, 2)                                   #隨機抽出兩個當父母-2
    parent1 = matrix[randomIndex[0],]
    parent2 = matrix[randomIndex[1],]
    kons = np.random.standard_normal(1)[0]                                      #論文中，equation(34)的固定參數
    for a in range(n+n):    #array/list的index從0開始
      if a < 9 :                                                                #前9個是DM的決策變數
        decision_dm_1 = mutation_step1(parent1[a] , parent2[a])
        temp_kid_decision_dm.append(decision_dm_1)
      elif a <18 :                                                              #第10~18個，是BP的決策變數
        decision_bp_1 = mutation_step1(parent1[a] , parent2[a])
        temp_kid_decision_bp.append(decision_bp_1)
      elif a <27 :                                                              #第19~27個，是DM的調節變數
        adjust_dm_1 = mutation_step2(parent1[a] , parent2[a])
        adjust_dm_2 = mutation_step3(adjust_dm_1, kons)
        kid_adjust_dm.append(adjust_dm_2)
      else :                                                                    #最後9個是BP的調節變數
        adjust_bp_1 = mutation_step2(parent1[a] , parent2[a])
        adjust_bp_2 = mutation_step3(adjust_bp_1, kons)
        kid_adjust_bp.append(adjust_bp_2)

    kid_decision_dm = mutation_step4(temp_kid_decision_dm , kid_adjust_dm)      #執行equation(35)-DM
    kid_decision_bp = mutation_step4(temp_kid_decision_bp , kid_adjust_bp)      #執行equation(35)-BP

    output_var = output_var + kid_decision_dm + kid_decision_bp + kid_adjust_dm + kid_adjust_bp

  output_matrix = np.array(output_var)
  output_matrix = output_matrix.reshape([kid_size, n+n])                        #從list轉換成array/矩陣
  return output_matrix

# Function C
### Use Hedged Problem objective function to determine the best offspring to constitute the population for the next generation

In [0]:
def hedge_step2BP(array_matrix,er):
  a_set_revenue=[]
  for c in range(array_matrix.shape[0]):
    an_option_revenue = 500*array_matrix[c]*(max(putBP[c,0] - 1.234*(1+er/100),0) - putBP[c,1])                #每個子代總共有9組解，對應到9個賣權選項的比例，所以unhedged revenue就需要針對每個賣權算一次，總計9次
    a_set_revenue.append(an_option_revenue)
  return sum(a_set_revenue)

In [0]:
def hedge_step2DM(array_matrix,er):
  a_set_revenue=[]
  for c in range(array_matrix.shape[0]):
    an_option_revenue = 500*array_matrix[c]*(max(putDM[c,0] - 0.6513*(1+er/100),0) - putDM[c,1])          #每個子代總共有9組解，對應到9個賣權選項的比例，所以unhedged revenue就需要針對每個賣權算一次，總計9次
    a_set_revenue.append(an_option_revenue)
  return sum(a_set_revenue)

In [0]:
def hedge_step1(array_matrix):
  temp_var=[]
  for b in range(ExRate.shape[0]):
    urDM = 645*0.6513*(1+ExRate[b,0]/100)
    urBP = 272*1.234*(1+ExRate[b,1]/100)
    hrDM_temp = hedge_step2DM(array_matrix[0:9],ExRate[b,0])
    hrBP_temp = hedge_step2BP(array_matrix[9:18],ExRate[b,1])
    each_sim_revenue = urDM + urBP + hrDM_temp + hrBP_temp
    temp_var.append(each_sim_revenue)                                           #總共有1000個匯率變化值，所以每個子代會有1000個profit
  temp_matrix = np.array(temp_var)                                              #先換成一維矩陣
  output = len(temp_matrix[temp_matrix < 706])                                  #找出所有profit中，有多少數量利益小於706的
  return output

In [0]:
def objective_function(kid_matrix):
  compare_var=[]
  output_next_gen_matrix = np.ones((pop_size,n+n))
  for a in range(kid_matrix.shape[0]):                                          
    count_small_706 = hedge_step1(kid_matrix[a,])                               #每個子代都算出profit小於706的次數，總共有kid_size個
    compare_var.append(count_small_706)

  next_gen_index = heapq.nsmallest(pop_size, range(len(compare_var)), key=compare_var.__getitem__)       #找出profit小於706的次數最少的pop_size個
  for k in range(len(next_gen_index)):
    output_next_gen_matrix[k,] = kid_matrix[next_gen_index[k],]                 #將優良的解變成下一代的母代matrix           

  output_best_set = kid_matrix[next_gen_index[0],0:18]
  output_min_count = min(compare_var)
  return output_next_gen_matrix, output_best_set, output_min_count

# Function D
### Random select a couple solutions of the last generation with a specific ratio and random generate the rest solutions to be the next-phase first parent solution

In [0]:
def turn_next_phase(better_matrix):
  next_phase_var = []
  number_of_pickup = int(pop_size*next_phase_ratio)                             #依照比率算出有幾個要變成下一階段的起始母代
  indexList = range(pop_size)                                                   #隨機挑選的步驟-1
  randomIndex = random.sample(indexList, number_of_pickup)                      #隨機挑選的步驟-2
  for a in range(len(randomIndex)):
    for b in better_matrix[randomIndex[a],]:                                    #這邊應該可以優化，但我先用我會的方式解決
      next_phase_var.append(b)
  for c in range(pop_size-number_of_pickup):
    next_phase_var = next_phase_var + generate_a_solution()
  output_matrix = np.array(next_phase_var)
  output_matrix = output_matrix.reshape([pop_size, n+n])
  return output_matrix

# Adjust the paremeters to find the minimum chance of "revenue less than USD $ 706 million"

In [0]:
pop_size = n*3                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 30            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.05          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00021776 0.10576932 0.76040463 0.12271304 0.00058187 0.00267029
 0.00252265 0.00279122 0.00232923 0.00854976 0.67921648 0.04690944
 0.02778888 0.00480619 0.03459122 0.01071286 0.17811605 0.00930912]
第10代的最小值 : 23
第10代結束
第20代的最佳解 : [0.00000459 0.00179954 0.9970224  0.00003841 0.00000018 0.00000466
 0.00000132 0.0011289  0.         0.00352929 0.64879565 0.01447579
 0.0002436  0.00007076 0.00239231 0.00017841 0.00552519 0.324789  ]
第20代的最小值 : 12
第20代結束
第30代的最佳解 : [0.00000338 0.00093465 0.99816121 0.00003214 0.00001516 0.00002857
 0.00000178 0.00002967 0.00079344 0.01563419 0.50989399 0.15672376
 0.00618744 0.00813008 0.00322975 0.00330515 0.05534368 0.24155193]
第30代的最小值 : 13
第30代結束
第1階段結束


第10代的最佳解 : [0.00000016 0.00073669 0.99921095 0.00000025 0.00000016 0.00000017
 0.00000057 0.00005077 0.0000003  0.00582619 0.63839139 0.03381589
 0.00272264 0.0023661  0.00176471 0.00004169 0.00465171 0.31041968]
第10代的最小值 : 13
第10代結束
第20代的最佳解 : [0.00007418 0.00002509 0.99119981 0.00001115

In [0]:
pop_size = n*3                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 30            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.05          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00154111 0.79146218 0.0665603  0.09477829 0.02765341 0.00035594
 0.01636098 0.00044018 0.00084761 0.08539989 0.50563317 0.03457415
 0.01604109 0.00091912 0.01966661 0.0202428  0.31674609 0.00077707]
第10代的最小值 : 28
第10代結束
第20代的最佳解 : [0.0050679  0.3327658  0.65785277 0.00037832 0.00009344 0.00359372
 0.00016634 0.00000478 0.00007694 0.01911593 0.61754944 0.0440382
 0.00441745 0.00436275 0.02903918 0.05998062 0.19804802 0.02344841]
第20代的最小值 : 16
第20代結束
第30代的最佳解 : [0.00012139 0.00254103 0.9972697  0.00000344 0.00000065 0.00000002
 0.00000008 0.00001541 0.00004826 0.01238686 0.62579021 0.00155044
 0.00272699 0.00105841 0.00015129 0.02602915 0.3134535  0.01685315]
第30代的最小值 : 15
第30代結束
第1階段結束


第10代的最佳解 : [0.00083272 0.00882148 0.99032956 0.00000755 0.00000031 0.00000013
 0.00000536 0.00000036 0.00000253 0.01160646 0.61777625 0.01393897
 0.01690498 0.00811065 0.07490313 0.00060353 0.0335534  0.22260265]
第10代的最小值 : 14
第10代結束
第20代的最佳解 : [0.00036284 0.0589636  0.94011404 0.00012136 

In [0]:
pop_size = n*3                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 30            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.05          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00036906 0.50073748 0.43738622 0.03683642 0.00446894 0.01790259
 0.00069628 0.00017383 0.00142917 0.00366542 0.35731328 0.3595454
 0.01012631 0.05394275 0.00032296 0.04018099 0.16478032 0.01012257]
第10代的最小值 : 28
第10代結束
第20代的最佳解 : [0.00237764 0.00003607 0.99617478 0.00060411 0.00054162 0.00014559
 0.00000454 0.00001021 0.00010545 0.02886478 0.42051719 0.21084461
 0.0006359  0.00002102 0.0002512  0.05316761 0.28210282 0.00359488]
第20代的最小值 : 18
第20代結束
第30代的最佳解 : [0.00026835 0.11913632 0.88052821 0.00002081 0.00000355 0.0000005
 0.00002284 0.00001915 0.00000027 0.00831189 0.50014577 0.19377075
 0.00734159 0.00056596 0.00310136 0.08529923 0.00236958 0.19909387]
第30代的最小值 : 14
第30代結束
第1階段結束


第10代的最佳解 : [0.00039504 0.00829635 0.98917444 0.00003201 0.00000675 0.00208856
 0.00000036 0.00000649 0.         0.00065976 0.62150553 0.07909143
 0.00101438 0.00008164 0.00683245 0.01625238 0.25393208 0.02063034]
第10代的最小值 : 14
第10代結束
第20代的最佳解 : [0.00003003 0.00002737 0.99994031 0.00000002 0

In [0]:
pop_size = n*3                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 30            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.05          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00004936 0.01553772 0.97468859 0.00414809 0.00350658 0.00004456
 0.00002759 0.00002898 0.00196853 0.00897451 0.68198572 0.01727122
 0.00315741 0.00795575 0.00027957 0.25449302 0.00182292 0.02405988]
第10代的最小值 : 18
第10代結束
第20代的最佳解 : [0.00412418 0.26468813 0.72843889 0.00163568 0.00080676 0.00007998
 0.00014074 0.00007595 0.0000097  0.10155653 0.39353058 0.2036989
 0.01514425 0.01192473 0.02485289 0.08326768 0.00706233 0.1589621 ]
第20代的最小值 : 18
第20代結束
第30代的最佳解 : [0.0000002  0.00669802 0.97238045 0.00312746 0.01773075 0.00000522
 0.00000967 0.00000164 0.00004659 0.00376516 0.58270757 0.05287887
 0.06627893 0.00480042 0.10802864 0.00207061 0.01855226 0.16091753]
第30代的最小值 : 14
第30代結束
第1階段結束


第10代的最佳解 : [0.00000001 0.00038933 0.99809521 0.00051286 0.00080314 0.00000099
 0.00012728 0.00000091 0.00007028 0.00043055 0.59101845 0.14484321
 0.00010842 0.00307026 0.0025525  0.00667995 0.00543961 0.24585704]
第10代的最小值 : 16
第10代結束
第20代的最佳解 : [0.00000003 0.03767035 0.96106637 0.00116317 

In [0]:
pop_size = n*3                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 30            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.05          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00002328 0.17368982 0.80277862 0.01411869 0.00706066 0.00014249
 0.00110641 0.00052262 0.00055742 0.00551373 0.5700199  0.21164379
 0.00021985 0.01284291 0.02996299 0.00016688 0.00098204 0.16864792]
第10代的最小值 : 29
第10代結束
第20代的最佳解 : [0.         0.00000639 0.99996432 0.00000586 0.00000741 0.00000047
 0.00000202 0.00000025 0.00001328 0.0046008  0.57351402 0.11423326
 0.00152491 0.00818943 0.01432029 0.03457702 0.14740212 0.10163815]
第20代的最小值 : 12
第20代結束
第30代的最佳解 : [0.00000067 0.00227238 0.99538825 0.00080071 0.00000485 0.00007303
 0.00109888 0.00028772 0.00007352 0.0759839  0.5762051  0.00369186
 0.02926109 0.00550086 0.02505497 0.00810436 0.00919679 0.26700109]
第30代的最小值 : 14
第30代結束
第1階段結束


第10代的最佳解 : [0.00000001 0.00003053 0.99976645 0.00000165 0.00000137 0.00000219
 0.00019506 0.00000271 0.00000003 0.00141604 0.53079905 0.18003115
 0.00061313 0.00009502 0.00348968 0.00339161 0.1644326  0.11573173]
第10代的最小值 : 13
第10代結束
第20代的最佳解 : [0.         0.00000003 0.99998389 0.00001557

In [0]:
pop_size = n*3                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 30            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.05          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00000546 0.00016896 0.99821467 0.00068085 0.00038388 0.00009667
 0.00026011 0.00008622 0.0001032  0.00370129 0.43898756 0.2443044
 0.03536716 0.00141072 0.00017348 0.27464029 0.00106516 0.00034993]
第10代的最小值 : 15
第10代結束
第20代的最佳解 : [0.00000354 0.00245622 0.99560525 0.00042896 0.00009797 0.00099039
 0.00000605 0.00000123 0.00041039 0.00170827 0.65573072 0.00021508
 0.00807657 0.00024332 0.00047322 0.00103382 0.00073446 0.33178455]
第20代的最小值 : 13
第20代結束
第30代的最佳解 : [0.0000382  0.00239715 0.99686249 0.00040837 0.00000659 0.00022417
 0.0000078  0.00005482 0.00000041 0.02204925 0.58526079 0.08591528
 0.00017238 0.00821768 0.0001756  0.01823033 0.26014263 0.01983605]
第30代的最小值 : 12
第30代結束
第1階段結束


第10代的最佳解 : [0.         0.00011133 0.99974496 0.00000118 0.00000007 0.0000018
 0.00007154 0.00006909 0.00000002 0.00194278 0.55380595 0.17251237
 0.00003511 0.00035182 0.00002868 0.00546711 0.20950977 0.0563464 ]
第10代的最小值 : 13
第10代結束
第20代的最佳解 : [0.00117653 0.00192701 0.98250087 0.00128881 0

In [0]:
pop_size = n*3                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 30            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.05          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00009671 0.9927682  0.00155345 0.00110746 0.00225971 0.00169477
 0.00005867 0.00018807 0.00027297 0.01712327 0.55741941 0.04671054
 0.05646566 0.02401999 0.01392351 0.25604603 0.00433433 0.02395728]
第10代的最小值 : 25
第10代結束
第20代的最佳解 : [0.00012115 0.00294614 0.99654301 0.00002521 0.0000018  0.00034849
 0.00000504 0.0000014  0.00000777 0.06135079 0.55721049 0.07616721
 0.00683778 0.03370455 0.00023687 0.00011498 0.1801052  0.08427214]
第20代的最小值 : 14
第20代結束
第30代的最佳解 : [0.0000001  0.0004647  0.99813542 0.00021925 0.00004096 0.0000088
 0.00018086 0.00094977 0.00000013 0.00067392 0.57259599 0.12503743
 0.02983991 0.000164   0.00228701 0.0545066  0.14916288 0.06573224]
第30代的最小值 : 14
第30代結束
第1階段結束


第10代的最佳解 : [0.00037302 0.18002005 0.81886334 0.00002657 0.00069363 0.00000458
 0.00001502 0.00000054 0.00000326 0.00000417 0.64524139 0.02545529
 0.00010311 0.08691343 0.00079356 0.00006061 0.23928945 0.00213898]
第10代的最小值 : 16
第10代結束
第20代的最佳解 : [0.00000012 0.05987812 0.94008558 0.0000002  

In [0]:
pop_size = n*3                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 30            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.05          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00000471 0.01453056 0.9807989  0.00028817 0.00010615 0.00286278
 0.00006872 0.00000285 0.00133717 0.00291302 0.49831257 0.18171624
 0.02572095 0.00944106 0.09632644 0.00635026 0.01821467 0.16100479]
第10代的最小值 : 16
第10代結束
第20代的最佳解 : [0.00000199 0.00050757 0.99923132 0.00006676 0.00000804 0.00003082
 0.00007917 0.00001206 0.00006227 0.00242851 0.54335277 0.16051858
 0.00846332 0.00003612 0.00103929 0.24506096 0.00085144 0.03824901]
第20代的最小值 : 15
第20代結束
第30代的最佳解 : [0.00001484 0.00014826 0.99983135 0.00000041 0.00000001 0.00000034
 0.000004   0.00000071 0.00000009 0.00178589 0.51816227 0.155069
 0.01726921 0.00028014 0.0079111  0.03293391 0.01024042 0.25634806]
第30代的最小值 : 13
第30代結束
第1階段結束


第10代的最佳解 : [0.00000595 0.00001096 0.99985811 0.00000024 0.00000658 0.00000081
 0.00009173 0.00000115 0.00002446 0.00963763 0.59630641 0.03013014
 0.08296613 0.00290833 0.05219492 0.01616654 0.15896603 0.05072388]
第10代的最小值 : 14
第10代結束
第20代的最佳解 : [0.00000028 0.00001037 0.99980772 0.0000013  0

In [0]:
pop_size = n*3                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 30            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.05          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00041773 0.01423076 0.9684915  0.00060997 0.00765142 0.00018198
 0.0070171  0.00006069 0.00133886 0.00048077 0.46610681 0.24903142
 0.0049285  0.00031464 0.00452374 0.09055904 0.00043158 0.1836235 ]
第10代的最小值 : 16
第10代結束
第20代的最佳解 : [0.00000011 0.00023971 0.99795053 0.00001524 0.00174532 0.00000004
 0.00001522 0.00002893 0.00000489 0.00148078 0.68713354 0.00914463
 0.00000119 0.00097779 0.0059673  0.00877772 0.18456239 0.10195466]
第20代的最小值 : 14
第20代結束
第30代的最佳解 : [0.00000004 0.00004231 0.99993066 0.0000014  0.00000193 0.00001328
 0.00000045 0.00000957 0.00000035 0.00068471 0.63749997 0.01829205
 0.01155631 0.00152497 0.00511705 0.00029292 0.31942417 0.00560786]
第30代的最小值 : 13
第30代結束
第1階段結束


第10代的最佳解 : [0.00000086 0.00177914 0.99800041 0.00003995 0.00009302 0.00006124
 0.00001075 0.00001173 0.0000029  0.0008019  0.62917413 0.01869963
 0.00118999 0.0037307  0.0000124  0.00003452 0.00005165 0.34630508]
第10代的最小值 : 13
第10代結束
第20代的最佳解 : [0.00000514 0.02406037 0.97575747 0.00000087

In [0]:
pop_size = n*3                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 30            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.05          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.0004634  0.16615183 0.79201468 0.02939007 0.0010209  0.00093988
 0.00234699 0.00650508 0.00116716 0.21769259 0.46506015 0.0063483
 0.02751826 0.00145116 0.01181501 0.01150247 0.03601767 0.22259438]
第10代的最小值 : 21
第10代結束
第20代的最佳解 : [0.00000228 0.00416035 0.98758517 0.00045632 0.00075139 0.00005429
 0.00664581 0.0003329  0.00001149 0.06112517 0.4551634  0.16693932
 0.02800094 0.0000211  0.00228811 0.02125141 0.00647739 0.25873316]
第20代的最小值 : 14
第20代結束
第30代的最佳解 : [0.00008316 0.13514394 0.85830471 0.00057596 0.00458156 0.00011076
 0.00017795 0.00098807 0.00003389 0.0108116  0.58869374 0.07755202
 0.00905104 0.00325303 0.01663402 0.05251677 0.10221702 0.13927075]
第30代的最小值 : 13
第30代結束
第1階段結束


第10代的最佳解 : [0.0000092  0.00275171 0.99704564 0.00000166 0.00009336 0.00000001
 0.00000153 0.00009671 0.00000017 0.00212218 0.56927178 0.11494741
 0.00098345 0.00061074 0.00489366 0.0139282  0.08571795 0.20752461]
第10代的最小值 : 12
第10代結束
第20代的最佳解 : [0.00008544 0.00135208 0.99827777 0.00000426 

In [0]:
pop_size = n*3                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 30            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.05          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00034648 0.95447871 0.03881309 0.00243177 0.00011072 0.00012744
 0.00132396 0.00189562 0.00047222 0.00070978 0.50288789 0.08251473
 0.05498746 0.01441706 0.00465358 0.0022011  0.00252222 0.33510618]
第10代的最小值 : 26
第10代結束
第20代的最佳解 : [0.00005959 0.68538613 0.30673075 0.00056042 0.00016601 0.00276906
 0.00171768 0.00002873 0.00258163 0.00377161 0.56787773 0.09025023
 0.00097547 0.00043715 0.00021819 0.00009558 0.24573039 0.09064365]
第20代的最小值 : 18
第20代結束
第30代的最佳解 : [0.00011175 0.14989411 0.84880122 0.00000113 0.00095898 0.00001062
 0.00000071 0.00003959 0.00018188 0.0134863  0.55507455 0.10672781
 0.00305142 0.01083546 0.00783743 0.0486623  0.00262169 0.25170304]
第30代的最小值 : 14
第30代結束
第1階段結束


第10代的最佳解 : [0.00000001 0.09222759 0.90211261 0.00002641 0.0037717  0.0000101
 0.00127828 0.00016978 0.00040351 0.0033921  0.43409213 0.26515762
 0.00342004 0.00025915 0.01016387 0.00072853 0.009874   0.27291257]
第10代的最小值 : 15
第10代結束
第20代的最佳解 : [0.00000016 0.00912908 0.99084777 0.00000238 

In [0]:
pop_size = n*3                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 30            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.05          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00026806 0.00071398 0.99873525 0.00005697 0.00000076 0.00004465
 0.00001071 0.0000048  0.00016481 0.00454974 0.42225613 0.2596857
 0.00608202 0.01540285 0.00018901 0.00266864 0.02315531 0.2660106 ]
第10代的最小值 : 15
第10代結束
第20代的最佳解 : [0.00027566 0.01276598 0.97817077 0.00272588 0.00047586 0.00496198
 0.00000138 0.00022449 0.00039801 0.02549557 0.52624828 0.13239284
 0.00299258 0.0014147  0.00427956 0.04146912 0.23658329 0.02912406]
第20代的最小值 : 13
第20代結束
第30代的最佳解 : [0.00000063 0.00082265 0.99904132 0.00000129 0.00012204 0.00000045
 0.00000067 0.00000048 0.00001047 0.01525894 0.56141658 0.12559264
 0.0043698  0.00455249 0.00105161 0.00602556 0.01942282 0.26230956]
第30代的最小值 : 11
第30代結束
第1階段結束


第10代的最佳解 : [0.00014426 0.00032716 0.99760565 0.00165625 0.0000049  0.00002075
 0.00002351 0.00011592 0.00010159 0.00187891 0.49612247 0.22348
 0.02402578 0.00153618 0.03156121 0.00403308 0.19262037 0.02474199]
第10代的最小值 : 16
第10代結束
第20代的最佳解 : [0.00001297 0.00046643 0.99096322 0.00543367 0.0

In [0]:
pop_size = n*3                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 30            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.05          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00002471 0.00016921 0.99885059 0.00046617 0.00008198 0.0000279
 0.00010943 0.00016631 0.00010369 0.00445046 0.54392905 0.11942394
 0.0016295  0.00006306 0.00204365 0.30489164 0.00102598 0.02254272]
第10代的最小值 : 15
第10代結束
第20代的最佳解 : [0.00000208 0.0031859  0.99639677 0.00014113 0.00007639 0.0000159
 0.00005071 0.00012598 0.00000514 0.00159756 0.63268143 0.03424097
 0.01801417 0.0045042  0.05461444 0.00245758 0.22429856 0.0275911 ]
第20代的最小值 : 14
第20代結束
第30代的最佳解 : [0.00000018 0.00005551 0.99993454 0.00000518 0.00000054 0.00000001
 0.00000003 0.00000329 0.00000072 0.00375643 0.60029394 0.11217185
 0.00521265 0.00068869 0.01109162 0.0001789  0.02481246 0.24179345]
第30代的最小值 : 14
第30代結束
第1階段結束


第10代的最佳解 : [0.         0.00196871 0.9978081  0.00000252 0.00000043 0.00001594
 0.00000007 0.00020323 0.00000101 0.00269884 0.63927268 0.012336
 0.00013204 0.00129796 0.02671375 0.0002267  0.06391524 0.25340679]
第10代的最小值 : 13
第10代結束
第20代的最佳解 : [0.         0.00011855 0.9995633  0.00000223 0.0

In [0]:
pop_size = n*3                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 30            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.05          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00025572 0.00506012 0.98357241 0.0000831  0.00921322 0.00168125
 0.0000096  0.00001501 0.00010956 0.0166309  0.5874858  0.10732851
 0.01946109 0.00244046 0.02298676 0.09927361 0.01720416 0.12718871]
第10代的最小值 : 17
第10代結束
第20代的最佳解 : [0.00000037 0.52474128 0.474922   0.00000812 0.00000348 0.00024669
 0.00001038 0.0000642  0.00000348 0.00186924 0.54322803 0.14717628
 0.06112257 0.00415556 0.00087926 0.0005166  0.00990553 0.23114693]
第20代的最小值 : 17
第20代結束
第30代的最佳解 : [0.00001982 0.02832449 0.96100616 0.00007049 0.00042374 0.01012842
 0.00000675 0.00001982 0.00000032 0.00432479 0.67107795 0.00041295
 0.00001767 0.00001124 0.00003164 0.00067229 0.01145497 0.31199651]
第30代的最小值 : 13
第30代結束
第1階段結束


第10代的最佳解 : [0.00000209 0.000063   0.99987963 0.00003857 0.00000221 0.00000807
 0.         0.00000189 0.00000453 0.01293817 0.50713772 0.1794152
 0.00341359 0.00000201 0.00009404 0.02889535 0.03654088 0.23156304]
第10代的最小值 : 13
第10代結束
第20代的最佳解 : [0.00002228 0.00200666 0.99526037 0.00041094 

In [0]:
pop_size = n*3                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 30            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.05          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00026795 0.02179412 0.93838556 0.00798183 0.00399972 0.02622039
 0.00036952 0.00091104 0.00006986 0.08849863 0.47979558 0.09796928
 0.05766794 0.00937327 0.0180312  0.00920631 0.21579502 0.02366277]
第10代的最小值 : 18
第10代結束
第20代的最佳解 : [0.00002003 0.00381058 0.98424264 0.00104778 0.00638293 0.00263101
 0.00115259 0.00003811 0.00067434 0.06845724 0.46633889 0.19339862
 0.00025568 0.00707918 0.0180352  0.21192671 0.01241876 0.02208972]
第20代的最小值 : 19
第20代結束
第30代的最佳解 : [0.00004988 0.00020179 0.98961597 0.00276166 0.00283139 0.001618
 0.00033261 0.00228187 0.00030683 0.0100674  0.57438348 0.0938958
 0.00016134 0.00008171 0.00408114 0.00060622 0.31642195 0.00030097]
第30代的最小值 : 13
第30代結束
第1階段結束


第10代的最佳解 : [0.00000107 0.01034491 0.98913806 0.00000233 0.00000323 0.00000025
 0.00050984 0.00000007 0.00000022 0.00035619 0.46237605 0.20260847
 0.0090708  0.00082469 0.00626505 0.31807245 0.00002772 0.00039859]
第10代的最小值 : 18
第10代結束
第20代的最佳解 : [0.00000475 0.09153059 0.90790088 0.00000008 0.

In [0]:
pop_size = n*3                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 30            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.05          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00011677 0.11886165 0.82062745 0.02541957 0.00006137 0.0089965
 0.00343524 0.01646204 0.0060194  0.0092586  0.67264635 0.03889694
 0.00008151 0.00427415 0.00093119 0.00140866 0.02662713 0.24587547]
第10代的最小值 : 19
第10代結束
第20代的最佳解 : [0.00074413 0.01166461 0.97549189 0.00892467 0.00264545 0.00042607
 0.00003333 0.00000067 0.00006919 0.0354995  0.54405953 0.14682703
 0.00276144 0.0477925  0.00091212 0.1501407  0.00556701 0.06644017]
第20代的最小值 : 19
第20代結束
第30代的最佳解 : [0.00000078 0.00010709 0.99987632 0.00000239 0.00000976 0.00000124
 0.00000106 0.00000127 0.00000008 0.00208233 0.67035692 0.0296803
 0.00050023 0.00022712 0.03193682 0.01272554 0.05744255 0.19504817]
第30代的最小值 : 14
第30代結束
第1階段結束


第10代的最佳解 : [0.00001666 0.00004166 0.99980033 0.00010156 0.00000061 0.0000192
 0.00001621 0.00000355 0.00000023 0.00026517 0.63177328 0.05000758
 0.00489248 0.00020281 0.00001037 0.01332071 0.0006806  0.29884699]
第10代的最小值 : 12
第10代結束
第20代的最佳解 : [0.00000817 0.01324948 0.98369915 0.00211262 0.

In [0]:
pop_size = n*3                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 30            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.05          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00046737 0.22194802 0.75885921 0.00182774 0.00664729 0.00762844
 0.00194522 0.00008385 0.00059287 0.12818643 0.51535772 0.04555712
 0.00578895 0.0008123  0.01932559 0.03584279 0.24894445 0.00018465]
第10代的最小值 : 19
第10代結束
第20代的最佳解 : [0.00017681 0.92930837 0.05953598 0.00808855 0.00146788 0.00008887
 0.00002823 0.00103002 0.00027528 0.01105656 0.65833152 0.04199805
 0.00406543 0.00086403 0.00300508 0.00011098 0.03823593 0.24233242]
第20代的最小值 : 19
第20代結束
第30代的最佳解 : [0.00129033 0.37689907 0.61652246 0.00167514 0.0029754  0.00059173
 0.00002954 0.00000102 0.00001531 0.04044324 0.55475328 0.07460675
 0.00469735 0.00060803 0.00728587 0.00143308 0.30381605 0.01235635]
第30代的最小值 : 14
第30代結束
第1階段結束


第10代的最佳解 : [0.00048373 0.83257327 0.16637577 0.000146   0.00003641 0.00006317
 0.00001121 0.00001299 0.00029745 0.13023272 0.47470985 0.07480958
 0.00228812 0.00932576 0.03261209 0.00745816 0.05927402 0.2092897 ]
第10代的最小值 : 18
第10代結束
第20代的最佳解 : [0.00000345 0.37636043 0.62357981 0.00000926

In [0]:
pop_size = n*3                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 30            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.05          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

In [0]:
pop_size = n*3                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 30            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.05          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

In [0]:
pop_size = n*3                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 30            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.05          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

In [0]:
#以下是可以調的參數
pop_size = n*2                  #預設7
kid_size = pop_size*2           #預設7
max_generation = 40            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.1          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 1            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00011548 0.47603202 0.18862939 0.32162388 0.00007354 0.01207935
 0.00080675 0.00000278 0.00063681 0.00529763 0.56175389 0.03261535
 0.03779388 0.00045834 0.00200172 0.02348746 0.31941895 0.01717278]
第10代的最小值 : 29
第10代結束
第20代的最佳解 : [0.00001254 0.03222582 0.96588632 0.00105517 0.00004221 0.00002226
 0.00001341 0.00007479 0.00066749 0.0248983  0.63202819 0.00998819
 0.01668295 0.00163184 0.00035801 0.01069206 0.00038467 0.3033358 ]
第20代的最小值 : 14
第20代結束
第30代的最佳解 : [0.00002038 0.00001601 0.99973981 0.00000079 0.00000025 0.00002137
 0.00018906 0.00000004 0.00001228 0.01938415 0.62271627 0.07570827
 0.00366747 0.03915895 0.00176328 0.03057734 0.00198961 0.20503468]
第30代的最小值 : 17
第30代結束
第40代的最佳解 : [0.00000477 0.00053414 0.99907289 0.00008611 0.00009809 0.00012862
 0.00001781 0.00005375 0.00000382 0.04759729 0.34827564 0.33411285
 0.00309999 0.00802158 0.00801204 0.11886318 0.00910857 0.12290886]
第40代的最小值 : 21
第40代結束
第1階段結束


第10代的最佳解 : [0.00008722 0.00005765 0.99337542 0.00004311

In [0]:
#以下是可以調的參數
pop_size = n*2                  #預設7
kid_size = pop_size*2           #預設7
max_generation = 30            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.1          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 6            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00005804 0.02428512 0.94646014 0.00482943 0.00096415 0.00277099
 0.00233471 0.01521628 0.00308114 0.02378422 0.59921816 0.2335589
 0.0018216  0.1055057  0.00304771 0.00397741 0.00122239 0.0278639 ]
第10代的最小值 : 39
第10代結束
第20代的最佳解 : [0.00001292 0.00040177 0.99854267 0.00030169 0.00019983 0.00002817
 0.00031294 0.00011889 0.00008113 0.00022605 0.44775904 0.28755018
 0.00925007 0.00082388 0.00019788 0.02447582 0.22825911 0.00145796]
第20代的最小值 : 18
第20代結束
第30代的最佳解 : [0.00004627 0.00352887 0.99481497 0.00145819 0.00004327 0.0000493
 0.00000066 0.00002448 0.000034   0.0003056  0.45936168 0.2283927
 0.0014055  0.06006348 0.00168286 0.00137617 0.22464842 0.02276361]
第30代的最小值 : 15
第30代結束
第1階段結束


第10代的最佳解 : [0.00000007 0.00028602 0.99960717 0.00008521 0.00000144 0.00000118
 0.0000004  0.00000001 0.0000185  0.02475554 0.52944659 0.1848949
 0.01790852 0.00604326 0.00018368 0.00138694 0.22325259 0.01212798]
第10代的最小值 : 18
第10代結束
第20代的最佳解 : [0.00000018 0.0056326  0.9669028  0.02708907 0.0

In [0]:
#以下是可以調的參數
pop_size = n*2                  #預設7
kid_size = pop_size*2           #預設7
max_generation = 30            #預設100代
max_phase = 2                   #預設5個階段
next_phase_ratio = 0.1          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

1.0
1.0
本次最佳解 : [5.45119135e-10 7.90333306e-06 9.99989811e-01 9.85201104e-11
 3.98905830e-10 6.80650168e-07 1.36651409e-08 1.58998215e-06
 2.55780192e-10 3.28373849e-02 6.26899637e-01 2.45355371e-02
 6.53232457e-04 9.23781828e-03 2.60817558e-02 6.19128297e-03
 1.44843115e-01 1.28720236e-01]
本次最大值 : 13
總共跑了幾秒231.2690029144287


In [0]:
#以下是可以調的參數
pop_size = n*3                  #預設7
kid_size = pop_size*2           #預設7
max_generation = 40            #預設100代
max_phase = 2                   #預設5個階段
next_phase_ratio = 0.1          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

1.0
0.9999999999999998
本次最佳解 : [3.44720611e-08 5.37274911e-07 9.99637201e-01 4.54670884e-06
 8.97315887e-05 1.36863236e-07 2.12777382e-08 2.98459898e-07
 2.67492176e-04 9.31752830e-02 5.49102380e-01 3.09612083e-03
 1.47711091e-04 2.91775958e-02 1.95057476e-02 2.31783537e-01
 6.37527721e-04 7.33740970e-02]
本次小於706次數的最小值 : 19
總共跑了幾秒461.6925141811371


In [0]:
#以下是可以調的參數
pop_size = n*3                  #預設7
kid_size = pop_size*2           #預設7
max_generation = 40            #預設100代
max_phase = 2                   #預設5個階段
next_phase_ratio = 0.1          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00046585 0.90326551 0.04299834 0.01468555 0.00343096 0.00929851
 0.00276635 0.00006992 0.02301901 0.00127206 0.5242041  0.23191076
 0.00020665 0.00329807 0.00055339 0.00790747 0.23063285 0.00001466]
第10代的最小值 : 31
第10代結束
第20代的最佳解 : [0.00478845 0.82259038 0.0033614  0.16544739 0.00080725 0.00142889
 0.00087773 0.00047087 0.00022764 0.36030655 0.30422607 0.05523101
 0.00098751 0.00012019 0.01702488 0.05731707 0.02884293 0.17594377]
第20代的最小值 : 33
第20代結束
第30代的最佳解 : [0.00419269 0.9742872  0.00512233 0.00123406 0.0008617  0.0127805
 0.00113802 0.00014184 0.00024165 0.0170518  0.58346375 0.0187785
 0.00016487 0.01262482 0.00360102 0.00947149 0.35468804 0.00015571]
第30代的最小值 : 22
第30代結束
第40代的最佳解 : [0.00000007 0.98133841 0.01793973 0.00010774 0.00000117 0.00059017
 0.00001215 0.00000386 0.0000067  0.06811294 0.52402662 0.11126708
 0.0001463  0.01961794 0.2111369  0.04967713 0.01205557 0.00395954]
第40代的最小值 : 27
第40代結束
第1階段結束


第10代的最佳解 : [0.00000474 0.90913797 0.09081494 0.00001663 0

In [0]:
#以下是可以調的參數
pop_size = n*5                  #預設7
kid_size = pop_size*2           #預設7
max_generation = 40            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.05          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00087563 0.0613786  0.85470194 0.01538772 0.05498633 0.0015721
 0.00090616 0.00521502 0.00497652 0.03948777 0.66751479 0.02041093
 0.00683479 0.03734221 0.0056487  0.00831265 0.10716747 0.10728068]
第10代的最小值 : 23
第10代結束
第20代的最佳解 : [0.00433959 0.07279364 0.91476257 0.00007972 0.00271763 0.00068259
 0.00376716 0.00009138 0.00076571 0.0142222  0.53005261 0.14674275
 0.00015689 0.01210412 0.00191351 0.03708693 0.00140694 0.25631404]
第20代的最小值 : 13
第20代結束
第30代的最佳解 : [0.00031873 0.01073894 0.98643393 0.00067863 0.00126251 0.00003775
 0.00039804 0.0000986  0.00003287 0.0023327  0.51604436 0.15396155
 0.01866814 0.00392989 0.00090849 0.02272595 0.00313292 0.27829601]
第30代的最小值 : 13
第30代結束
第40代的最佳解 : [0.00000076 0.04338758 0.95562896 0.00028143 0.00037154 0.00005013
 0.00017748 0.00001331 0.00008879 0.00527838 0.6320024  0.03710904
 0.01972031 0.07898253 0.00014178 0.10032038 0.11883997 0.0076052 ]
第40代的最小值 : 17
第40代結束
第1階段結束


第10代的最佳解 : [0.00000326 0.00564144 0.98997742 0.00023642 

In [0]:
#以下是可以調的參數
pop_size = n*2                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 40            #預設100代
max_phase = 3                   #預設5個階段
next_phase_ratio = 0.1          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 2             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00000007 0.21761105 0.77389629 0.00175838 0.00000907 0.00324671
 0.00343627 0.00001357 0.0000286  0.00008026 0.26613976 0.44676646
 0.00002168 0.02094724 0.26101702 0.00142427 0.00101718 0.00258612]
第10代的最小值 : 35
第10代結束
第20代的最佳解 : [0.         0.00000046 0.99998674 0.00000006 0.         0.00000299
 0.00000007 0.00000004 0.00000965 0.03235979 0.52912295 0.00949512
 0.18174786 0.00195581 0.00757907 0.14423522 0.00032743 0.09317675]
第20代的最小值 : 22
第20代結束
第30代的最佳解 : [0.         0.0001403  0.99985894 0.00000006 0.00000001 0.00000016
 0.00000002 0.0000005  0.00000001 0.0000082  0.61399792 0.07692141
 0.00050903 0.00033847 0.00000771 0.00057309 0.16538349 0.14226068]
第30代的最小值 : 11
第30代結束
第40代的最佳解 : [0.         0.00000009 0.9999999  0.         0.         0.
 0.         0.         0.         0.00000023 0.60451294 0.10676277
 0.00000011 0.00001261 0.00000014 0.28807517 0.00036364 0.0002724 ]
第40代的最小值 : 14
第40代結束
第1階段結束


第10代的最佳解 : [0.         0.00000011 0.99999989 0.         0.     

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = pop_size*2           #預設7
max_generation = 30            #預設100代
max_phase = 2                   #預設5個階段
next_phase_ratio = 0.3          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.0000878  0.05902472 0.9212716  0.00074611 0.00118649 0.00080846
 0.00002503 0.00242377 0.01442601 0.01254388 0.73176431 0.04060001
 0.0108528  0.00338998 0.00269418 0.04259852 0.03850743 0.1170489 ]
第10代的最小值 : 25
第10代結束
第20代的最佳解 : [0.00024987 0.99650335 0.00017987 0.00005377 0.00019119 0.00119135
 0.00004982 0.00031076 0.00127002 0.09981574 0.39274932 0.25270482
 0.00120252 0.00944067 0.00998531 0.02003763 0.03059301 0.18347098]
第20代的最小值 : 27
第20代結束
第30代的最佳解 : [0.00000544 0.99997609 0.00000065 0.0000005  0.00000041 0.00000014
 0.00000633 0.00000838 0.00000206 0.00495461 0.32417509 0.47948705
 0.00100641 0.00514968 0.09577037 0.00556443 0.01034262 0.07354975]
第30代的最小值 : 33
第30代結束
第1階段結束


第10代的最佳解 : [0.00000859 0.8280398  0.16527975 0.00006057 0.00578706 0.00026896
 0.00005075 0.00004169 0.00046282 0.06915932 0.43466955 0.14400077
 0.00223199 0.00038597 0.00850306 0.00605906 0.28109485 0.05389542]
第10代的最小值 : 21
第10代結束
第20代的最佳解 : [0.00000096 0.99879079 0.0002749  0.00000026

In [0]:
#以下是可以調的參數
pop_size = n*5                  #預設7
kid_size = pop_size*1           #預設7
max_generation = 40            #預設100代
max_phase = 1                   #預設5個階段
next_phase_ratio = 0.1          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 2             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.0000045  0.00000017 0.99963378 0.00010414 0.00011557 0.
 0.00000581 0.00013599 0.00000004 0.00000085 0.00002812 0.9779357
 0.00038751 0.0000087  0.00000001 0.00000672 0.00003463 0.02159776]
第10代的最小值 : 50
第10代結束
第20代的最佳解 : [0.00020682 0.00000012 0.98081787 0.00001255 0.00007199 0.00012338
 0.01865285 0.00007832 0.0000361  0.40278149 0.02985092 0.22584074
 0.33856738 0.00207659 0.00006873 0.0000052  0.00076893 0.00004001]
第20代的最小值 : 52
第20代結束
第30代的最佳解 : [0.         0.00000005 0.97313323 0.00000001 0.02538883 0.
 0.0014773  0.00000034 0.00000024 0.00001101 0.62719737 0.0011185
 0.01712541 0.11590223 0.00000199 0.00000625 0.00001251 0.23862473]
第30代的最小值 : 18
第30代結束
第40代的最佳解 : [0.00000094 0.         0.98020318 0.         0.         0.
 0.01979588 0.         0.         0.00000169 0.536626   0.01450708
 0.00017478 0.4423727  0.00000476 0.         0.00000133 0.00631166]
第40代的最小值 : 36
第40代結束
第1階段結束


1.0
1.0000000000000002
本次最佳解 : [0.00000094 0.         0.98020318 0.         0.   

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = pop_size*5           #預設7
max_generation = 30            #預設100代
max_phase = 1                   #預設5個階段
next_phase_ratio = 0.1          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 2             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00000034 0.13876634 0.86055696 0.00000267 0.00060585 0.00000002
 0.0000027  0.00000626 0.00005887 0.00050072 0.42042187 0.32169582
 0.0060972  0.00020723 0.0132744  0.01936653 0.06735088 0.15108535]
第10代的最小值 : 20
第10代結束
第20代的最佳解 : [0.00000001 0.01189537 0.98809548 0.00000002 0.00000605 0.00000129
 0.00000005 0.         0.00000171 0.00168744 0.6360851  0.02655563
 0.00012162 0.00018071 0.00001317 0.00001268 0.01368426 0.32165939]
第20代的最小值 : 13
第20代結束
第30代的最佳解 : [0.         0.00698708 0.99250818 0.00000043 0.00004245 0.00000019
 0.00000005 0.0000003  0.00046131 0.00660147 0.73527594 0.00014587
 0.00000248 0.00244255 0.00001064 0.22626034 0.00070924 0.02855147]
第30代的最小值 : 19
第30代結束
第1階段結束


1.0
0.9999999999999999
本次最佳解 : [0.         0.00698708 0.99250818 0.00000043 0.00004245 0.00000019
 0.00000005 0.0000003  0.00046131 0.00660147 0.73527594 0.00014587
 0.00000248 0.00244255 0.00001064 0.22626034 0.00070924 0.02855147]
本次小於706次數的最小值 : 19
總共跑了幾秒136.23487448692322


In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = pop_size*3           #預設7
max_generation = 50            #預設100代
max_phase = 1                   #預設5個階段
next_phase_ratio = 0.1          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, min_count = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最小值 : {}".format(gen+1, min_count))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次小於706次數的最小值 : {}".format(min_count))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00154482 0.08512165 0.87366163 0.00396928 0.00476567 0.0050138
 0.00972991 0.0001189  0.01607435 0.13314023 0.53683655 0.09166264
 0.00244491 0.01678022 0.00029313 0.11172592 0.10299194 0.00412445]
第10代的最小值 : 29
第10代結束
第20代的最佳解 : [0.00001831 0.01552868 0.98272051 0.00072313 0.00015522 0.0007722
 0.0000257  0.00000574 0.00005051 0.10959357 0.48302492 0.09675803
 0.0047731  0.0242731  0.00521411 0.03160527 0.12452706 0.12023084]
第20代的最小值 : 15
第20代結束
第30代的最佳解 : [0.00012442 0.00081279 0.99845626 0.00042479 0.00001524 0.00005219
 0.00011188 0.00000215 0.00000027 0.02510265 0.54001241 0.08843656
 0.16469423 0.00094762 0.00552711 0.08940237 0.07062154 0.01525551]
第30代的最小值 : 24
第30代結束
第40代的最佳解 : [0.00000409 0.00009965 0.99732255 0.00007264 0.00228632 0.00018768
 0.00002582 0.00000102 0.00000023 0.14424132 0.49052545 0.04251676
 0.02214767 0.01893332 0.02052915 0.18570732 0.07395623 0.00144277]
第40代的最小值 : 18
第40代結束
第50代的最佳解 : [0.0000005  0.00005496 0.99704922 0.00000228 0.00267959